In [25]:
%load_ext autoreload
%autoreload 2
import sys
import keras
from keras.models import Model
from keras.layers import Conv2D, UpSampling2D
sys.path.insert(0, "../scripts/")
from load_encoder_decoder import build_encoder_decoder_from_vgg
from data_generator import DataGenerator
from losses import overall_loss_wrapper
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import multi_gpu_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
PATIENCE=30
PATH_MODEL_CHECKPOINTS = "../model_checkpoints"
PATH_LOGS = "../logs"

BATCH_SIZE=16

In [2]:
inputs, model = build_encoder_decoder_from_vgg()

In [36]:
path_combined_train = "D:\\Image Matting Dataset\\merged_ds_two\\"
path_alphas_train = "D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Training_set\\all_alphas\\"

path_combined_test = "" + path_combined_train
path_alphas_test = "" + path_alphas_train

In [37]:
train_gen = DataGenerator(path_combined_train, path_alphas_train, BATCH_SIZE, True)
test_gen = DataGenerator(path_combined_test, path_alphas_test, BATCH_SIZE, True) 

In [31]:
tensor_board = keras.callbacks.TensorBoard(log_dir=PATH_LOGS, histogram_freq=0, write_graph=True, write_images=True)
model_name = PATH_MODEL_CHECKPOINTS + 'encoder_decoder.{epoch:02d}-{val_loss:.4f}-{val_acc:.4f}.hdf5'
model_checkpoint = ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True)
early_stop = EarlyStopping('val_loss', patience=PATIENCE, verbose=1)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=PATIENCE // 4, verbose=1)

callbacks = [tensor_board, model_checkpoint, early_stop, reduce_lr]

In [16]:
model.compile(optimizer="adam", loss=overall_loss_wrapper(inputs), use_multiprocessing=True, workers=3, callbacks=callbacks)

In [24]:
history = model.fit_generator(gen)

Epoch 1/1
2/2 [==============================] - 56s 28s/step - loss: 819096.2500
